# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
M. Häberle  ->  M. Häberle  |  ['M. Häberle']
C. Clontz  ->  C. Clontz  |  ['C. Clontz']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
J. Lian  ->  J. Lian  |  ['J. Lian']
U. Chadayammuri  ->  U. Chadayammuri  |  ['U. Chadayammuri']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']


C. Gapp  ->  C. Gapp  |  ['C. Gapp']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
Arxiv has 64 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2406.01676


extracting tarball to tmp_2406.01676...

 done.


H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
Retrieving document from  https://arxiv.org/e-print/2406.01688


...

/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2227/1211882699.py:51: LatexWarning: 2406.01809 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


 done.


C. Gapp  ->  C. Gapp  |  ['C. Gapp']


Found 75 bibliographic references in tmp_2406.02179/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2406.02305


extracting tarball to tmp_2406.02305... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.01706-b31b1b.svg)](https://arxiv.org/abs/2406.01706) | **The Extremely Metal Rich Knot of Stars at the Heart of the Galaxy**  |
|| <mark>H.-W. Rix</mark>, et al. -- incl., <mark>A. Pillepich</mark>, <mark>J. Lian</mark> |
|*Appeared on*| *2024-06-05*|
|*Comments*| *11 pages, 7 figures, submitted to ApJ*|
|**Abstract**|            We show with Gaia XP spectroscopy that extremely metal-rich stars in the Milky Way (EMR; $[M/H]_{XP} > 0.5$) - but only those - are largely confined to a tight "knot" at the center of the Galaxy. This EMR knot is round in projection, has a fairly abrupt edge near $\sim 1.5$kpc, and is a dynamically hot system. This central knot also contains very metal-rich (VMR; $+0.2\le [M/H]_{XP} \le +0.4$) stars. However, in contrast to EMR stars, the bulk of VMR stars form an extended, highly flattened distribution in the inner Galaxy ($R_{\mathrm{GC}}\lesssim 5$ kpc). We draw on TNG50 simulations of Milky Way analogs for context and find that compact, metal-rich knots confined to $<1.5$kpc are a universal feature. In typical simulated analogs, the top 5-10% most metal-rich stars are confined to a central knot; however, in our Milky Way data this fraction is only 0.1%. Dust-penetrating wide-area near-infrared spectroscopy, such as SDSS-V, will be needed for a rigorous estimate of the fraction of stars in the Galactic EMR knot. Why in our Milky Way only EMR giants are confined to such a central knot remains to be explained. Remarkably, the central few kiloparsecs of the Milky Way harbor both the highest concentration of metal-poor stars (the `poor old heart') and almost all EMR stars. This highlights the stellar population diversity at the bottom of galactic potential wells.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.02179-b31b1b.svg)](https://arxiv.org/abs/2406.02179) | **Abundances of trace constituents in Jupiter's atmosphere inferred from Herschel/PACS observations**  |
|| <mark>C. Gapp</mark>, et al. |
|*Appeared on*| *2024-06-05*|
|*Comments*| *Accepted for publication in Astronomy and Astrophysics, 16 pages, 4 tables, 24 figures incl. Appendix*|
|**Abstract**|            $Context.$ On October 31, 2009, the Photodetector Array Camera and Spectrometer (PACS) on board the Herschel Space Observatory observed far-infrared spectra of Jupiter between 50 and 220$\,\mu$m as part of the program "Water and Related Chemistry in the Solar System". $Aims.$ We investigate the disk-averaged chemical composition of Jupiter's atmosphere as a function of height using these observations. $Methods.$ We used the Planetary Spectrum Generator (PSG) and the least-squares fitting technique to infer the abundances of trace constituents. $Results.$ The PACS data include numerous spectral lines attributable to ammonia (NH$_3$), methane (CH$_4$), phosphine (PH$_3$), water (H$_2$O), and deuterated hydrogen (HD) in the Jovian atmosphere. We infer an ammonia abundance profile that decreases from a mole fraction of $(1.7\pm 0.8)\times 10^{-4}$ at $p\sim 900\,$mbar to $(1.7\pm 0.9)\times 10^{-8}$ at $p\sim 275\,$mbar, following a fractional scale height of about 0.114. For phosphine, we find a mole fraction of $(7.2\pm 1.2)\times 10^{-7}$ at pressures higher than $(550\pm 100)\,$mbar and a decrease of its abundance at lower pressures following a fractional scale height of $(0.09\pm 0.02)$. Our analysis delivers a methane mole fraction of $(1.49\pm 0.09)\times 10^{-3}$. Analyzing the HD $R(0)$ line at $112.1\,\mu$m yields a new measurement of Jupiter's D/H ratio, $\text{D/H}=(1.5\pm 0.6)\times 10^{-5}$. Finally, the PACS data allow us to put the most stringent $3\sigma$ upper limits yet on the mole fractions of hydrogen halides in the Jovian troposphere. These new upper limits are $<1.1\times 10^{-11}$ for hydrogen fluoride (HF), $<6.0\times 10^{-11}$ for hydrogen chloride (HCl), $<2.3\times 10^{-10}$ for hydrogen bromide (HBr) and $<1.2\times 10^{-9}$ for hydrogen iodide (HI) and support the proposed condensation of hydrogen halides into ammonium halide salts in the Jovian troposphere.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.01707-b31b1b.svg)](https://arxiv.org/abs/2406.01707) | **X-ray bright AGN in local dwarf galaxies: insights from eROSITA**  |
|| A. Sacchi, A. Bogdan, <mark>U. Chadayammuri</mark>, A. Ricarte |
|*Appeared on*| *2024-06-05*|
|*Comments*| *17 pages, 12 figures, 1 table. Submitted to ApJ. Comments are welcome*|
|**Abstract**|            Although supermassive black holes (SMBHs) reside in the heart of virtually every massive galaxy, it remains debated whether dwarf galaxies also commonly host SMBHs. Because low-mass galaxies may retain a memory of the assembly history of their black holes, probing the black hole occupation fraction of local dwarf galaxies might offer insights into the growth and seeding mechanisms of the first black holes. In this work, we exploit the Western half of the eROSITA all-sky survey (covering $20,000~\rm{deg^2}$) and compile a catalog of accreting SMBHs in local ($D<200$~Mpc) dwarf galaxies. After cleaning our sample from cosmic X-ray background sources, X-ray binaries, and ultraluminous X-ray sources, we identify 74 AGN-dwarf galaxy pairs. Using this large and uniform sample, we derive a luminosity function of dwarf galaxy AGN, fitting it with a power law function and obtaining ${\rm d}N/{\rm d}L_{\rm X} = (15.9\pm2.2)\times L_{\rm X}^{-1.63\pm0.05}$. Measuring the offset between the centroid of dwarf galaxies and the X-ray sources, we find that about $50\%$ of the AGN are likely off-nuclear, in agreement with theoretical predictions. We also compare the black hole-to-stellar mass relation of the AGN in our sample with the local and high-redshift relations, finding that our sources better adhere to the former. This suggests that local AGN across different mass scales underwent a similar growth history. Finally, we compare our sources with semi-analytical models: while our sample is too shallow to distinguish between different seeding models, it favors a growth mechanism linked to the star-formation rate of the host galaxy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.01688-b31b1b.svg)](https://arxiv.org/abs/2406.01688) | **oMEGACat III. Multi-band photometry and metallicities reveal spatially well-mixed populations within $\omega$ Centauri's half-light radius**  |
|| M. S. Nitschai, et al. -- incl., <mark>N. Neumayer</mark>, <mark>M. Häberle</mark>, <mark>C. Clontz</mark> |
|*Appeared on*| *2024-06-05*|
|*Comments*| *22 pages, 18 figures, and 3 tables. Accepted for publication in ApJ*|
|**Abstract**|            $\omega$ Centauri, the most massive globular cluster in the Milky Way, has long been suspected to be the stripped nucleus of a dwarf galaxy that fell into the Galaxy a long time ago. There is considerable evidence for this scenario including a large spread in metallicity and an unusually large number of distinct sub-populations seen in photometric studies. In this work, we use new MUSE spectroscopic and HST photometric catalogs to investigate the underlying metallicity distributions as well as the spatial variations of the populations within the cluster up to its half-light radius. Based on 11,050 member stars, the [M/H] distribution has a median of $ (-1.614 \pm 0.003)$ dex and a large spread of $\sim$ 1.37 dex reaching from $ -0.67$ dex to $ -2.04$ dex for 99.7 % of the stars. In addition, we show the chromosome map of the cluster, which separates the red giant branch stars into different sub-populations, and analyze the sub-populations of the metal-poorest component. Finally, we do not find any metallicity gradient within the half-light radius, and the different sub-populations are well mixed.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.02305-b31b1b.svg)](https://arxiv.org/abs/2406.02305) | **Debris Disks can Contaminate Mid-Infrared Exoplanet Spectra: Evidence for a Circumstellar Debris Disk around Exoplanet Host WASP-39**  |
|| L. Flagg, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-06-05*|
|*Comments*| *accepted to ApJL*|
|**Abstract**|            The signal from a transiting planet can be diluted by astrophysical contamination. In the case of circumstellar debris disks, this contamination could start in the mid-infrared and vary as a function of wavelength, which would then change the observed transmission spectrum for any planet in the system. The MIRI/LRS WASP-39b transmission spectrum shows an unexplained dip starting at $\sim$10 $\mu$m that could be caused by astrophysical contamination. The spectral energy distribution displays excess flux at similar levels to that which are needed to create the dip in the transmission spectrum. In this article, we show that this dip is consistent with the presence of a bright circumstellar debris disk, at a distance of $>$2 au. We discuss how a circumstellar debris disk like that could affect the atmosphere of WASP-39b. We also show that even faint debris disks can be a source of contamination in MIRI exoplanet spectra.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.01809-b31b1b.svg)](https://arxiv.org/abs/2406.01809) | **Use the 4S (Signal-Safe Speckle Subtraction): Explainable Machine Learning reveals the Giant Exoplanet AF Lep b in High-Contrast Imaging Data from 2011**  |
|| M. J. Bonse, et al. -- incl., <mark>E. Matthews</mark> |
|*Appeared on*| *2024-06-05*|
|*Comments*| *Submitted to AJ, 26 pages, 15 figures, comments welcome, code available on ReadtheDocs: this https URL*|
|**Abstract**|            The main challenge of exoplanet high-contrast imaging (HCI) is to separate the signal of exoplanets from their host stars, which are many orders of magnitude brighter. HCI for ground-based observations is further exacerbated by speckle noise originating from perturbations in the Earth's atmosphere and imperfections in the telescope optics. Various data post-processing techniques are used to remove this speckle noise and reveal the faint planet signal. Often, however, a significant part of the planet signal is accidentally subtracted together with the noise. In the present work, we use explainable machine learning to investigate the reason for the loss of the planet signal for one of the most used post-processing methods: Principal Component Analysis (PCA). We find that PCA learns the shape of the telescope point spread function for high numbers of PCA components. This representation of the noise captures not only the speckle noise, but also the characteristic shape of the planet signal. Building upon these insights, we develop a new post-processing method (4S) that constrains the noise model to minimize this signal loss. We apply our model to 11 archival HCI datasets from the VLT-NACO instrument in the L'-band and find that our model consistently outperforms PCA. The improvement is largest at close separations to the star ($\leq 4 \lambda /D$) providing up to 1.5 magnitudes deeper contrast. This enhancement enables us to detect the exoplanet AF Lep b in data from 2011, 11 years before its subsequent discovery. We present updated orbital parameters for this object.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.01676-b31b1b.svg)](https://arxiv.org/abs/2406.01676) | **All-Sky Kinematics of the Distant Halo: The Reflex Response to the LMC**  |
|| V. Chandra, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-06-05*|
|*Comments*| *25 pages, 15 figures. Submitted to ApJ*|
|**Abstract**|            The infall of the Large Magellanic Cloud (LMC) is predicted to displace the inner Milky Way (MW), imprinting an apparent 'reflex motion' on the observed velocities of distant halo stars. We construct the largest all-sky spectroscopic dataset of luminous red giant stars from $50-160$ kpc, including a new survey of the southern celestial hemisphere. We fit the full 6D kinematics of our data to measure the amplitude and direction of the inner MW's motion towards the outer halo. The observed velocity grows with distance such that, relative to halo stars at $100$ kpc, the inner MW is lurching at $\approx 40$ km s$^{-1}$ towards a recent location along the LMC's past orbit. Our measurements align with N-body simulations of the halo's response to a $1.8 \times 10^{11} M_\odot$ LMC on first infall, suggesting that the LMC is at least 15% as massive as the MW. Our findings highlight the dramatic disequilibrium of the MW outskirts, and will enable more accurate measurements of the total mass of our Galaxy.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \e at position 37</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2406.01706.md
    + _build/html/tmp_2406.01706/./metal_rich_on_sky.png
    + _build/html/tmp_2406.01706/./RGC_vs_MH.png
    + _build/html/tmp_2406.01706/./MHrich-core-fraction.png
    + _build/html/tmp_2406.01706/./EMR_l_density_profile.png
exported in  _build/html/2406.02179.md
    + _build/html/tmp_2406.02179/./figures/hf_results.png
    + _build/html/tmp_2406.02179/./figures/ph3_lines.png
    + _build/html/tmp_2406.02179/./figures/nh3_contours.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\MH}{\ensuremath{{[\mathrm{M}/\mathrm{H}]}}}$
$\newcommand{\apc}[1]{ #1}$
$\newcommand{\thing}{knot~}$
$\newcommand{\thingns}{knot}$
$\newcommand{\things}{knots~}$
$\newcommand{\Thing}{Knot~}$
$\newcommand{\Thingns}{Knot}$
$\newcommand{\MHXP}{\ensuremath{\mathrm{[M/H]}_{\mathrm{XP}}}}$</div>



<div id="title">

# The Extremely Metal Rich $\Thing$ of Stars at the Heart of the Galaxy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.01706-b31b1b.svg)](https://arxiv.org/abs/2406.01706)<mark>Appeared on: 2024-06-05</mark> -  _11 pages, 7 figures, submitted to ApJ_

</div>
<div id="authors">

<mark>H.-W. Rix</mark>, et al. -- incl., <mark>A. Pillepich</mark>, <mark>J. Lian</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ We show with Gaia XP spectroscopy that extremely metal-rich stars in the Milky Way (EMR; $\mathrm{[M/H]}_{\mathrm{XP}} \gtrsim 0.5$ ) -- but only those -- are largely confined to a tight "knot” at the center of the Galaxy.This EMR knot is round in projection, has a fairly abrupt edge near $\sim 1.5$ kpc, and is a dynamically hot system.This central knot also contains very metal-rich (VMR; $+0.2\le \mathrm{[M/H]}_{\mathrm{XP}} \le +0.4$ ) stars.However, in contrast to EMR stars, the bulk of VMR stars form an extended, highly flattened distribution in the inner Galaxy ( $R_{\mathrm{GC}}\lesssim 5$ kpc).We draw on TNG50 simulations of Milky Way analogs for context and find that compact, metal-rich knots confined to $\lesssim 1.5$ kpc are a universal feature.In typical simulated analogs, the top 5-10 \% most metal-rich stars are confined to a central knot; however, in our Milky Way data this fraction is only 0.1 \% .Dust-penetrating wide-area near-infrared spectroscopy, such as SDSS-V, will be needed for a rigorous estimate of the fraction of stars in the Galactic EMR knot.Why in our Milky Way only EMR giants are confined to such a central knot remains to be explained.Remarkably, the central few kiloparsecs of the Milky Way harbor both the highest concentration of metal-poor stars (the `poor old heart') and almost all EMR stars.This highlights the stellar population diversity at the bottom of galactic potential wells.

</div>

<div id="div_fig1">

<img src="tmp_2406.01706/./metal_rich_on_sky.png" alt="Fig6" width="100%"/>

**Figure 6. -** 
   On-sky density distribution of very (and extremely) metal-rich giant stars, with $\MHXP\ge +0.3$ from Gaia DR3 XP spectra, in three different $\MH$XP  bins.
   The top panel (containing $\sim 85\%$ of all these stars) shows a flattened, disk-like distribution, concentrated towards the inner Galaxy. This central concentration increases toward higher metallicity (middle panel).
  The bottom panel, showing the distribution of extremely metal-rich (EMR) stars with $\MHXP \gtrsim 0.5$, shows a striking change in morphology compared to the bin with $0.3<$\MH$XP < 0.4$: the distribution is dominated by a central *$\thing$* with a sharp cut-off at $R_{\mathrm{GC}} \sim 1.5$ kpc. The relative prominence of this *$\thing$* increases by an order of magnitude from the previous bin (see also Fig. \ref{fig:EMRprofile}). Note that in all three panels, the imprint of dust extinction is apparent at the lowest latitudes; and the bar may contribute to the slight $\pm l$ asymmetry. Foreground stars closer than 4 kpc from the Sun have been removed (see sample selection criteria at the end of Section \ref{sec:sample} (*fig:metal_rich_on_sky*)

</div>
<div id="div_fig2">

<img src="tmp_2406.01706/./RGC_vs_MH.png" alt="Fig8.1" width="50%"/><img src="tmp_2406.01706/./MHrich-core-fraction.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Radial extent of different mono-abundance stellar populations for Milky Way analogs from the TNG50 simulation \citep{Pillepich2023}. The left panel shows the radius $R_{\mathrm{GC}}$ that encloses 68\% of a mono-abundance population as a function of $\MH$ for a handful of example galaxies. This left panel shows that for (at least these) simulated galaxies the most metal-rich stars are very centrally concentrated, forming a $\thing$ of $\lesssim 1$ kpc. This is in qualitative agreement with our analysis of the Milky Way (see Fig. \ref{fig:D68percentile_MH}). However, in these simulated galaxies the central confinement encompasses stars within $\Delta$\MH$ = 0.4$ of the maximal metallicity, whereas in our Galaxy the stars in the $\thing$ are within 0.1 dex of the maximal metallicity.  To eliminate issues revolving around metallicity systematics, we also consider which fraction of the stars' MFD is part of this central $\thing$ns .
    The right panel shows, for a much larger set of TNG50 Milky Way analogs, the (most metal rich) fraction of the MDF that is confined to within  $R_{\mathrm{GC}}= 1.5$ kpc: typically between 5\% and 15\% of all stars within 20 kpc. By comparison, the *face value* fraction of stars in the Milky Way's $\thing$(i.e. in the Gaia XP sample as defined in Section \ref{sec:metal-rich-distribution}) is dramatically lower, only 0.1\%(Fig. \ref{fig:D68percentile_MH}). However, this observed fraction constitutes a lower limit, as the most centrally concentrated populations in the Milky Way are inevitably most extincted, and hence most underrepresented in the Gaia XP sample. (*fig:sims_extent*)

</div>
<div id="div_fig3">

<img src="tmp_2406.01706/./EMR_l_density_profile.png" alt="Fig1" width="100%"/>

**Figure 1. -** 
   The longitudinal number density profile of the very metal-rich stars towards the center of the Milky Way. These are the same stars as shown in Fig. \ref{fig:metal_rich_on_sky}, but integrated over Galactic latitude (excluding $\pm 3^\circ$ because of the severe dust extinction). The dashed line just illustrates a Gaussian of $\sigma=10^\circ$ centered on $|l|=0$ in order to illustrate the level of asymmetry induced by dust extinction. While for the least metal-rich bin ($0.3<$\MH$XP<0.4$) the density peak of the central $\thing$ is only $\sim 3\times$ above the surrounding (in projection) disk-like distribution, this contrast has grown to $100\times$ in the EMR bin ($\MHXP > 0.5$): EMR stars are basically only found in the $\lesssim 1$ kpc $\thing$ns. (*fig:EMRprofile*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.01706"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Abundances of trace constituents in Jupiter's atmosphere inferred from Herschel/PACS$\thanks{_ Herschel_ is an ESA space observatory with science instruments provided by European-led Principal Investigator consortia and with important participation from NASA.}$ observations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.02179-b31b1b.svg)](https://arxiv.org/abs/2406.02179)<mark>Appeared on: 2024-06-05</mark> -  _Accepted for publication in Astronomy and Astrophysics, 16 pages, 4 tables, 24 figures incl. Appendix_

</div>
<div id="authors">

<mark>C. Gapp</mark>, et al.

</div>
<div id="abstract">

**Abstract:** On October 31, 2009, the Photodetector Array Camera and Spectrometer (PACS) on board the _Herschel_ Space Observatory observed far-infrared spectra of Jupiter in the wavelength range between 50 and 220 $ \mu$ m as part of the program "Water and Related Chemistry in the Solar System". The spectra have an effective spectral resolution between 900 and 3500, depending on the wavelength and grating order. We investigate the disk-averaged chemical composition of Jupiter's atmosphere as a function of height using these observations. We used the Planetary Spectrum Generator (PSG) and the least-squares fitting technique to infer the abundances of trace constituents. The PACS data include numerous spectral lines attributable to ammonia ($NH_3$ ), methane ($CH_4$ ), phosphine ($PH_3$ ), water ($H_2$ O), and deuterated hydrogen (HD) in the Jovian atmosphere and probe the chemical composition from $p\sim 275$ mbar to $p\sim 900$ mbar. From the observations, we infer an ammonia abundance profile that decreases from a mole fraction of $(1.7\pm 0.8)\times 10^{-4}$ at $p\sim 900$ mbar to $(1.7\pm 0.9)\times 10^{-8}$ at $p\sim 275$ mbar, following a fractional scale height of about $0.114$ . For phosphine, we find a mole fraction of $(7.2\pm 1.2)\times 10^{-7}$ at pressures higher than $(550\pm 100)$ mbar and a decrease of its abundance at lower pressures following a fractional scale height of $(0.09\pm 0.02)$ . Our analysis delivers a methane mole fraction of $(1.49\pm 0.09)\times 10^{-3}$ . Analyzing the HD $R(0)$ line at $112.1 \mu$ m yields a new measurement of Jupiter's D/H ratio, $\text{D/H}=(1.5\pm 0.6)\times 10^{-5}$ . Finally, the PACS data allow us to put the most stringent $3\sigma$ upper limits yet on the mole fractions of hydrogen halides in the Jovian troposphere. These new upper limits are $<1.1\times 10^{-11}$ for hydrogen fluoride (HF), $<6.0\times 10^{-11}$ for hydrogen chloride (HCl), $<2.3\times 10^{-10}$ for hydrogen bromide (HBr) and $<1.2\times 10^{-9}$ for hydrogen iodide (HI) and support the proposed condensation of hydrogen halides into ammonium halide salts in the Jovian troposphere.

</div>

<div id="div_fig1">

<img src="tmp_2406.02179/./figures/hf_results.png" alt="Fig17" width="100%"/>

**Figure 17. -** Results of the upper limit determination of Jupiter's atmospheric hydrogen fluoride (\ce{HF}) mole fraction using the PACS data and the a priori forward model (Sec. \ref{sec:model_setup}) including the derived ammonia and phosphine profiles (Subsec. \ref{subsec:nh3} and \ref{subsec:ph3}) and the inferred methane mole fraction (Subsec. \ref{subsec:ch4}). Panels (a) to (c) show the PACS data around the wavelengths of three \ce{HF} lines in the PACS spectral range. These plots also show forward model spectra including \ce{HF} with the inferred upper limit mole fraction and without \ce{HF} at all. Panel (d) shows the results of the least-squares comparison between model spectra assuming different \ce{HF} mole fractions and the PACS data using $\Delta\chi_0^2$(see Equation \ref{eq:delta_chi-squared0}). The criterion for the upper limit ($\Delta\chi_0^2=9$) is shown with a horizontal blue dashed line and the mole fraction at which it is met is shown using a vertical blue dashed line. In total, 1000 values for the \ce{HF} mole fraction between 0 and $1\times 10^{-9}$ were used in the forward model to compare the resulting model spectra with the PACS data. (*fig:hf*)

</div>
<div id="div_fig2">

<img src="tmp_2406.02179/./figures/ph3_lines.png" alt="Fig9" width="100%"/>

**Figure 9. -** PACS data and model spectra of the five phosphine bands used for the analysis of phosphine in Jupiter's atmosphere as well as the \ce{PH_3}$R(5)$ band. The forward model spectra were synthesized using $a_{\ce{PH_3}}(\infty)=7.2\times 10^{-7}$, $p_{\ce{PH_3}}=550 $mbar, and $f_{\ce{PH_3}}=0.09$. (*fig:ph3_lines*)

</div>
<div id="div_fig3">

<img src="tmp_2406.02179/./figures/nh3_contours.png" alt="Fig5" width="100%"/>

**Figure 5. -** Least-squares comparison between the PACS data of the \ce{NH_3}$R(2)$, $R(5)$, and $R(7)$ bands and the model spectra synthesized using grids of ammonia abundance profiles expressed in $\Delta\chi^2$(see Equation \ref{eq:delta_chi-squared}). (*fig:nh3_contours*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.02179"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

84  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
